In [ ]:
!pip install --upgrade pip
!pip install scikit-image>=0.18.0
!pip install scikit-learn>=1.0.0

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import os
import glob
from pathlib import Path
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import tqdm
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.layers import LeakyReLU, GlobalAveragePooling2D
from keras.applications import VGG16
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


2025-05-28 10:52:34.056921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748429554.307919      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748429554.380265      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
np.random.seed(42)
random.seed(42)

In [ ]:
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU Available: True. Devices: {gpu_devices}")
else:
    print("GPU Available: False")

In [ ]:
# Configuration parameters - improved from reference solutions
class Config:
    # Multi-scale approach from rank 1 and rank 4 solutions
    SCALES = [0.4, 0.5]  # Multiple scales for robustness, reduced for demo
    PATCH_SIZE = 224  # Adjusted for common VGG16 input, rank 2 suggested larger
    BATCH_SIZE = 16  # Adjusted for memory constraints
    EPOCHS = 2  # Reduced significantly for quick demonstration
    LEARNING_RATE = 1e-4

    # Class names for better organization
    CLASS_NAMES = ['adult_males', 'subadult_males',
                   'adult_females', 'juveniles', 'pups']
    N_CLASSES = 5

    # Data paths - adjust if your data is located elsewhere
    DATA_ROOT_KAGGLE = "/kaggle/input/noaa-fisheries-steller-sea-lion-population-count/KaggleNOAASeaLions"
    # Assuming a local 'data' folder with Train, TrainDotted, Mismatched...
    DATA_ROOT_LOCAL = "data"

    if os.path.exists(DATA_ROOT_KAGGLE):
        DATA_ROOT = DATA_ROOT_KAGGLE
    elif os.path.exists(DATA_ROOT_LOCAL):
        DATA_ROOT = DATA_ROOT_LOCAL
    else:
        # Fallback if no data directory is found, notebook will use dummy data
        print("Warning: Data directories not found. Will use dummy data.")
        DATA_ROOT = "data_not_found"
        # Create dummy directories if they don't exist to prevent later errors
        os.makedirs(os.path.join(DATA_ROOT, "Train"), exist_ok=True)
        os.makedirs(os.path.join(DATA_ROOT, "TrainDotted"), exist_ok=True)
        # Create an empty mismatch file
        with open(os.path.join(DATA_ROOT, "MismatchedTrainImages.txt"), 'w') as f:
            pass

    TRAIN_DIR = os.path.join(DATA_ROOT, "Train")
    TRAIN_DOTTED_DIR = os.path.join(DATA_ROOT, "TrainDotted")
    MISMATCH_FILE = os.path.join(DATA_ROOT, "MismatchedTrainImages.txt")
    TEST_DIR = os.path.join(DATA_ROOT, "Test")  # For submission generation


config = Config()
print(f"Configuration loaded. Data root set to: {config.DATA_ROOT}")
print(
    f"Using scales: {config.SCALES}, Patch size: {config.PATCH_SIZE}, Epochs: {config.EPOCHS}")

In [7]:
def enhanced_get_data(filename, scale=0.4, patch_size=224):
    try:
        image_dotted_path = os.path.join(config.TRAIN_DOTTED_DIR, filename)
        image_original_path = os.path.join(config.TRAIN_DIR, filename)

        if not os.path.exists(image_dotted_path) or not os.path.exists(image_original_path):
            # print(f"Warning: Image files for {filename} not found.")
            return None, None

        image_dotted = cv2.imread(image_dotted_path)
        image_original = cv2.imread(image_original_path)

        if image_dotted is None or image_original is None:
            # print(f"Warning: Could not load image {filename}. Skipping.")
            return None, None

        img_blurred = cv2.GaussianBlur(image_dotted, (5, 5), 0)
        diff = cv2.absdiff(image_dotted, image_original)
        mask_gray = cv2.cvtColor(image_dotted, cv2.COLOR_BGR2GRAY)
        mask_bin = np.zeros_like(mask_gray)
        # Increased threshold for dot mask from 50 to 20 to catch more faint dots
        mask_bin[mask_gray > 20] = 255
        masked_diff = cv2.bitwise_or(diff, diff, mask=mask_bin)
        gray_diff = np.max(masked_diff, axis=2)

        blobs = skimage.feature.blob_log(
            gray_diff, min_sigma=2, max_sigma=7, num_sigma=3, threshold=0.01  # Adjusted blob params
        )

        h, w, _ = image_original.shape
        grid_w_count = int(np.ceil((w * scale) / patch_size))
        grid_h_count = int(np.ceil((h * scale) / patch_size))
        result_grid = np.zeros(
            (grid_h_count, grid_w_count, config.N_CLASSES), dtype='float32')

        for blob in blobs:
            y, x, sigma = map(int, blob)
            if 0 <= y < h and 0 <= x < w:
                b_val, g_val, r_val = img_blurred[y, x]
                grid_x_idx = min(
                    int((x * scale) // patch_size), grid_w_count - 1)
                grid_y_idx = min(
                    int((y * scale) // patch_size), grid_h_count - 1)

                cl = -1
                if r_val > 200 and g_val < 50 and b_val < 50:
                    cl = 0  # Adult Males (Red)
                elif r_val > 200 and g_val < 50 and b_val > 200:
                    cl = 1  # Subadult Males (Magenta)
                elif r_val < 50 and g_val > 150 and b_val < 50:
                    cl = 2  # Adult Females (Green)
                elif r_val < 50 and g_val < 50 and b_val > 150:
                    cl = 3  # Juveniles (Blue)
                elif r_val > 60 and r_val < 120 and g_val < 75 and b_val < 50:
                    cl = 4  # Pups (Brown)

                if cl != -1:
                    result_grid[grid_y_idx, grid_x_idx, cl] += 1

        valid_region_mask = (np.sum(image_dotted, axis=2) > 10).astype(
            'uint8')  # Mask out very dark regions
        valid_region_mask_bgr = cv2.cvtColor(
            valid_region_mask * 255, cv2.COLOR_GRAY2BGR)
        processed_img_masked = image_original * \
            (valid_region_mask_bgr // 255)  # Apply mask

        processed_img_resized = cv2.resize(
            processed_img_masked, (int(w * scale), int(h * scale)))
        processed_img_normalized = processed_img_resized.astype(
            'float32') / 255.0

        h_new, w_new, _ = processed_img_normalized.shape
        patches_x, patches_y = [], []

        for r_idx in range(grid_h_count):
            for c_idx in range(grid_w_count):
                y_start, y_end = r_idx * patch_size, (r_idx + 1) * patch_size
                x_start, x_end = c_idx * patch_size, (c_idx + 1) * patch_size

                # Ensure patch boundaries are within the scaled image dimensions
                y_end = min(y_end, h_new)
                x_end = min(x_end, w_new)

                # Check if the actual patch start is still valid (e.g. for last row/col)
                if y_start >= h_new or x_start >= w_new:
                    continue

                patch = processed_img_normalized[y_start:y_end,
                                                 x_start:x_end, :]
                actual_h, actual_w, _ = patch.shape

                # Pad if patch is smaller than target size (edges)
                if actual_h < patch_size or actual_w < patch_size:
                    pad_h = patch_size - actual_h
                    pad_w = patch_size - actual_w
                    # Pad with zeros (black)
                    patch = np.pad(patch, ((0, pad_h), (0, pad_w),
                                   (0, 0)), mode='constant', constant_values=0)

                # Ensure final shape is correct
                if patch.shape == (patch_size, patch_size, 3):
                    patches_x.append(patch)
                    patches_y.append(result_grid[r_idx, c_idx, :])

        if not patches_x:  # If no patches were created for this image
            return None, None

        return np.array(patches_x), np.array(patches_y)

    except Exception as e:
        print(f"Error processing {filename} at scale {scale}: {e}")
        import traceback
        # traceback.print_exc() # Uncomment for detailed error stack
        return None, None


def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))


def custom_rmse_loss(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.cast(y_true, tf.float32) - y_pred)))


print("Data processing functions defined.")

In [11]:
def create_enhanced_cnn(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), padding='same',
               input_shape=input_shape), BatchNormalization(), Activation('relu'),
        Conv2D(32, (3, 3), padding='same'), BatchNormalization(), Activation(
            'relu'), MaxPooling2D((2, 2)), Dropout(0.2),
        Conv2D(64, (3, 3), padding='same'), BatchNormalization(), Activation('relu'),
        Conv2D(64, (3, 3), padding='same'), BatchNormalization(), Activation(
            'relu'), MaxPooling2D((2, 2)), Dropout(0.3),
        Conv2D(128, (3, 3), padding='same'), BatchNormalization(
        ), Activation('relu'),
        Conv2D(128, (3, 3), padding='same'), BatchNormalization(
        ), Activation('relu'), MaxPooling2D((2, 2)), Dropout(0.4),
        Flatten(),
        Dense(512, activation='relu'), Dropout(0.5),
        # Linear activation for regression
        Dense(config.N_CLASSES, activation='linear')
    ])
    return model


def create_vgg_transfer_model(input_shape):
    base_model = VGG16(weights='imagenet', include_top=False,
                       input_shape=input_shape)
    base_model.trainable = False  # Freeze base layers initially

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(config.N_CLASSES, activation='linear')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model


def create_data_generator():
    return ImageDataGenerator(
        rotation_range=45,  # Increased rotation
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.15,  # Increased shift
        height_shift_range=0.15,
        zoom_range=0.2,  # Increased zoom
        brightness_range=[0.7, 1.3],  # Wider brightness range
        fill_mode='nearest')


print("Model architectures and data generator defined.")

In [14]:
def load_training_data(max_files=2, scales=None):  # Reduced max_files for demo
    if scales is None:
        scales = config.SCALES

    bad_files = set()
    if os.path.exists(config.MISMATCH_FILE):
        try:
            with open(config.MISMATCH_FILE, 'r') as f:
                bad_files = set(line.strip() for line in f if line.strip())
        except Exception as e:
            print(f"Could not read mismatch file: {e}")

    all_image_files = []
    if os.path.exists(config.TRAIN_DIR):
        all_image_files = [f for f in os.listdir(config.TRAIN_DIR)
                           if f.endswith('.jpg') and f not in bad_files]
        # Shuffle for variety if max_files is small
        random.shuffle(all_image_files)

    if not all_image_files:
        print("No training image files found. Using dummy data.")
        # Create dummy data if no real data is found
        dummy_X = np.random.rand(
            32, config.PATCH_SIZE, config.PATCH_SIZE, 3).astype('float32')
        dummy_y = np.random.randint(
            0, 2, (32, config.N_CLASSES)).astype('float32')
        return dummy_X, dummy_y

    files_to_process = all_image_files[:
                                       max_files] if max_files else all_image_files
    print(f"Loading data from {len(files_to_process)} files, scales: {scales}")

    X_all, y_all = [], []
    for scale_val in tqdm.tqdm(scales, desc="Processing Scales"):
        for filename in tqdm.tqdm(files_to_process, desc=f"Scale {scale_val} Files", leave=False):
            X_batch, y_batch = enhanced_get_data(
                filename, scale=scale_val, patch_size=config.PATCH_SIZE)
            if X_batch is not None and len(X_batch) > 0:
                X_all.extend(X_batch)
                y_all.extend(y_batch)

    if not X_all:
        print("No patches extracted from available files. Using dummy data.")
        dummy_X = np.random.rand(
            32, config.PATCH_SIZE, config.PATCH_SIZE, 3).astype('float32')
        dummy_y = np.random.randint(
            0, 2, (32, config.N_CLASSES)).astype('float32')
        return dummy_X, dummy_y

    X = np.array(X_all)
    y = np.array(y_all)

    print(f"Data loading complete. Total patches: {len(X)}")
    if len(X) > 0:
        print(f"X shape: {X.shape}, y shape: {y.shape}")
        print(f"y distribution (sum per class): {np.sum(y, axis=0)}")
    return X, y


print("Training data loading function defined.")

In [15]:
def train_model_wrapper(model, X_train, y_train, X_val, y_val, model_name="model"):
    # Ensure model is compiled before training
    if not model._is_compiled:
        model.compile(
            optimizer=Adam(learning_rate=config.LEARNING_RATE),
            loss=custom_rmse_loss,
            metrics=['mse', 'mae']
        )

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5,
                      restore_best_weights=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                          patience=3, min_lr=1e-7, verbose=1),
        ModelCheckpoint(f"{model_name}_best.keras",
                        monitor='val_loss', save_best_only=True, verbose=0)
    ]

    print(f"\nTraining {model_name}... Input shape: {X_train.shape}")
    # model.summary() # Can be verbose

    datagen = create_data_generator()
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=config.BATCH_SIZE),
        steps_per_epoch=max(1, len(X_train) // config.BATCH_SIZE),
        epochs=config.EPOCHS,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )
    # Load best weights saved by ModelCheckpoint
    if os.path.exists(f"{model_name}_best.keras"):
        model.load_weights(f"{model_name}_best.keras")
        print(f"Loaded best weights for {model_name} from checkpoint.")
    return model, history


def evaluate_model_performance(model, X_test, y_test, model_name="model"):
    y_pred_raw = model.predict(X_test, verbose=0)
    # Post-processing: Clip negative predictions and apply threshold (from Rank 1)
    # Ensure non-negative counts
    y_pred_processed = np.clip(y_pred_raw, 0, None)
    y_pred_thresholded = y_pred_processed * \
        (y_pred_processed > 0.3)  # Thresholding idea

    rmse_raw = rmse(y_test, y_pred_raw)
    rmse_processed = rmse(y_test, y_pred_processed)
    rmse_thresholded = rmse(y_test, y_pred_thresholded)

    print(f"\n--- {model_name} Evaluation Results ---")
    print(f"RMSE (Raw Predictions): {rmse_raw:.4f}")
    print(f"RMSE (Processed - Clipped): {rmse_processed:.4f}")
    print(
        f"RMSE (Processed - Clipped & Thresholded > 0.3): {rmse_thresholded:.4f}")

    true_counts = np.sum(y_test, axis=0)
    # Using rounded thresholded for final count comparison
    pred_counts_final = np.sum(np.round(y_pred_thresholded), axis=0)

    print("Class-wise Counts (True vs Predicted [Rounded Thresholded]):")
    for i, name in enumerate(config.CLASS_NAMES):
        print(
            f"{name:>15s}: True={true_counts[i]:<5.0f} Pred={pred_counts_final[i]:<5.0f} Diff={(pred_counts_final[i]-true_counts[i]):<5.0f}")
    return {'rmse': rmse_processed, 'rmse_thresholded': rmse_thresholded, 'predictions': y_pred_processed}


def plot_training_summary(history, model_name):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{model_name} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    if 'mae' in history.history:
        plt.plot(history.history['mae'], label='Train MAE')
        plt.plot(history.history['val_mae'], label='Val MAE')
        plt.title(f'{model_name} - MAE')
        plt.xlabel('Epoch')
        plt.ylabel('MAE')
        plt.legend()
    plt.tight_layout()
    plt.show()


print("Model training and evaluation functions defined.")

948


In [16]:
def run_main_pipeline():
    print("=== Starting Sea Lion Counting Pipeline ===")

    # 1. Load Data
    print("\n--- Step 1: Loading Training Data ---")
    # For demonstration, using very few files and only one scale initially
    X, y = load_training_data(max_files=config.EPOCHS,
                              scales=config.SCALES[:1])

    if X is None or len(X) == 0:
        print("Critical error: No training data loaded. Exiting pipeline.")
        return {}, {}, None, None  # Return empty dicts and None for data

    if len(X) < config.BATCH_SIZE * 2:  # Ensure enough data for train/val/test split and batching
        print(
            f"Warning: Very few samples ({len(X)}). Results may not be meaningful. Duplicating data for stability.")
        factor = (config.BATCH_SIZE * 2) // len(X) + 1
        X = np.concatenate([X] * factor, axis=0)
        y = np.concatenate([y] * factor, axis=0)
        print(f"Augmented sample count to {len(X)}")

    # 2. Split Data
    print("\n--- Step 2: Splitting Data ---")
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(
        # 0.25 * 0.8 = 0.2 for val
        X_train_full, y_train_full, test_size=0.25, random_state=42)
    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")

    trained_models = {}
    evaluation_results = {}
    input_shape = X_train.shape[1:]

    # 3. Train Enhanced CNN
    print("\n--- Step 3: Training Enhanced CNN ---")
    try:
        cnn_model_instance = create_enhanced_cnn(input_shape)
        cnn_model_instance, cnn_history = train_model_wrapper(
            cnn_model_instance, X_train, y_train, X_val, y_val, "Enhanced_CNN")
        trained_models["Enhanced_CNN"] = cnn_model_instance
        evaluation_results["Enhanced_CNN"] = evaluate_model_performance(
            cnn_model_instance, X_test, y_test, "Enhanced_CNN")
        plot_training_summary(cnn_history, "Enhanced_CNN")
    except Exception as e:
        print(f"Error training Enhanced CNN: {e}")
        # traceback.print_exc()

    # 4. Train VGG16 Transfer Model
    print("\n--- Step 4: Training VGG16 Transfer Model ---")
    try:
        vgg_model_instance = create_vgg_transfer_model(input_shape)
        vgg_model_instance, vgg_history = train_model_wrapper(
            vgg_model_instance, X_train, y_train, X_val, y_val, "VGG16_Transfer")
        trained_models["VGG16_Transfer"] = vgg_model_instance
        evaluation_results["VGG16_Transfer"] = evaluate_model_performance(
            vgg_model_instance, X_test, y_test, "VGG16_Transfer")
        plot_training_summary(vgg_history, "VGG16_Transfer")

        # Optional: Fine-tune VGG16 (if epochs are sufficient)
        if config.EPOCHS > 5:  # Only fine-tune if initial training was somewhat substantial
            print("\n--- Step 4b: Fine-tuning VGG16 Transfer Model ---")
            # Unfreeze some layers of VGG16
            for layer in vgg_model_instance.layers[-4:]:
                # Keep BN frozen as per best practices
                if not isinstance(layer, BatchNormalization):
                    layer.trainable = True
            vgg_model_instance.compile(
                # Lower LR for fine-tuning
                optimizer=Adam(learning_rate=config.LEARNING_RATE / 10),
                loss=custom_rmse_loss, metrics=['mse', 'mae']
            )
            vgg_model_instance, vgg_ft_history = train_model_wrapper(
                vgg_model_instance, X_train, y_train, X_val, y_val, "VGG16_FineTuned")
            # Overwrite or add as new
            trained_models["VGG16_FineTuned"] = vgg_model_instance
            evaluation_results["VGG16_FineTuned"] = evaluate_model_performance(
                vgg_model_instance, X_test, y_test, "VGG16_FineTuned")
            plot_training_summary(vgg_ft_history, "VGG16_FineTuned")
    except Exception as e:
        print(f"Error training VGG16 model: {e}")
        # traceback.print_exc()

    # 5. Ensemble Predictions (if multiple models trained successfully)
    if len(trained_models) > 1:
        print("\n--- Step 5: Creating Ensemble Predictions ---")
        ensemble_preds_list = []
        for model_name, model_instance in trained_models.items():
            # Use the processed predictions from evaluation_results if available
            if model_name in evaluation_results and 'predictions' in evaluation_results[model_name]:
                ensemble_preds_list.append(
                    evaluation_results[model_name]['predictions'])
            else:  # Fallback to predict again if needed
                pred = model_instance.predict(X_test, verbose=0)
                pred_clipped = np.clip(pred, 0, None)
                ensemble_preds_list.append(pred_clipped)

        if ensemble_preds_list:
            # Averaging ensemble
            avg_ensemble_pred = np.mean(ensemble_preds_list, axis=0)
            avg_ensemble_pred_thresholded = avg_ensemble_pred * \
                (avg_ensemble_pred > 0.3)

            ensemble_rmse_val = rmse(y_test, avg_ensemble_pred)
            ensemble_rmse_thresh_val = rmse(
                y_test, avg_ensemble_pred_thresholded)

            print(
                f"Ensemble (Average) RMSE (Processed): {ensemble_rmse_val:.4f}")
            print(
                f"Ensemble (Average) RMSE (Processed & Thresholded): {ensemble_rmse_thresh_val:.4f}")
            evaluation_results["Ensemble_Average"] = {
                'rmse': ensemble_rmse_val, 'rmse_thresholded': ensemble_rmse_thresh_val, 'predictions': avg_ensemble_pred}

    # 6. Final Summary
    print("\n=== Pipeline Execution Summary ===")
    if not evaluation_results:
        print("No models were successfully trained or evaluated.")
    else:
        for name, res in evaluation_results.items():
            print(f"{name:>20s}: RMSE (Processed) = {res.get('rmse', float('nan')):.4f}, RMSE (Thresholded) = {res.get('rmse_thresholded', float('nan')):.4f}")

        best_model_name = min(evaluation_results.items(), key=lambda x: x[1].get(
            'rmse_thresholded', float('inf')))[0]
        print(
            f"\nBest performing model (based on thresholded RMSE): {best_model_name}")

    return trained_models, evaluation_results, X_test, y_test


print("Main training pipeline function defined.")

In [17]:
# This is the main execution block for the notebook.
print("Starting the improved sea lion counting pipeline execution...")
print(f"Using {config.EPOCHS} epochs for training. This is a demo run.")

trained_models_dict = {}
eval_results_dict = {}
X_test_final, y_test_final = None, None

try:
    trained_models_dict, eval_results_dict, X_test_final, y_test_final = run_main_pipeline()

    print("\n=== FINAL EXECUTION STATUS ===")
    if trained_models_dict:
        print(
            f"Successfully trained {len(trained_models_dict)} model(s): {list(trained_models_dict.keys())}")
        # Save the best model (if any was deemed best)
        if eval_results_dict:
            best_model_key = min(eval_results_dict, key=lambda k: eval_results_dict[k].get(
                'rmse_thresholded', float('inf')))
            if best_model_key in trained_models_dict:
                trained_models_dict[best_model_key].save(
                    f"best_overall_model.keras")
                print(
                    f"Saved best model ({best_model_key}) as best_overall_model.keras")
    else:
        print("No models were trained successfully in this run.")

except Exception as e:
    print(f"An critical error occurred during pipeline execution: {e}")
    import traceback
    traceback.print_exc()
finally:
    print("Pipeline execution finished.")

891
